In [2]:
!wget https://github.com/DanteNoguez/gpt-wilde/raw/main/data/wilde.txt

--2023-01-21 03:32:22--  https://github.com/DanteNoguez/gpt-wilde/raw/main/data/wilde.txt
Resolving github.com (github.com)... 140.82.114.4
Connecting to github.com (github.com)|140.82.114.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/DanteNoguez/gpt-wilde/main/data/wilde.txt [following]
--2023-01-21 03:32:23--  https://raw.githubusercontent.com/DanteNoguez/gpt-wilde/main/data/wilde.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4818475 (4.6M) [text/plain]
Saving to: ‘wilde.txt’

wilde.txt           100%[===================>]   4.59M  --.-KB/s    in 0.08s   

2023-01-21 03:32:24 (61.2 MB/s) - ‘wilde.txt’ saved [4818475/4818475]



In [ ]:
with open('wilde.txt', 'r', encoding='utf-8') as f:
  text = f.read()

In [ ]:
len(text)

4763434

In [ ]:
chars = sorted(list(set(text)))
vocab_size = len(chars)
print(''.join(chars))
print(vocab_size)


 !&'()*,-./0123456789:;?ABCDEFGHIJKLMNOPQRSTUVWXYZ[\]abcdefghijklmnopqrstuvwxyz£ÀÆÈÉàáâäæçèéêëîïóôöùûüÿŒœ–—‘’“”
112


In [ ]:
#character level tokenizer
stoi = {ch:i for i,ch in enumerate(chars)} #lookup table, mapping from characters to integers
itos = {i:ch for i,ch in enumerate(chars)}
encode = lambda s: [stoi[c] for c in s] #encoder: take a string, output integers
decode = lambda l: ''.join([itos[i] for i in l]) #decoder: take integers, output string

In [ ]:
encode("hello there"), decode(encode("hello there"))

([61, 58, 65, 65, 68, 1, 73, 61, 58, 71, 58], 'hello there')

In [ ]:
import torch

#encoding dataset
data = torch.tensor(encode(text), dtype=torch.long)
print(data.shape, data.dtype)
print(data[:100]) # todo el texto encoded

torch.Size([4763434]) torch.int64
tensor([  0,  43,  56,  58,  67,  58,  10,   1, 107,   1,  25,   1,  42,  74,
         72,  72,  62,  54,  67,   1,  33,  67,  67,  10,   0,   0,   0,   0,
         36,  54,  71,  60,  58,   1,  57,  68,  68,  71,   1,  68,  69,  58,
         67,  62,  67,  60,   1,  68,  67,   1,  72,  67,  68,  76,  78,   1,
         65,  54,  67,  57,  72,  56,  54,  69,  58,   1,  54,  73,   1,  55,
         54,  56,  64,   1,  68,  59,   1,  72,  73,  54,  60,  58,  10,   0,
          0,   0,   0,  40,  58,  73,  58,  71,   1,  43,  54,  55,  68,  74,
         71,  68])


In [ ]:
#split data into train and validation sets
n = int(0.9*len(data)) #90% train, 10%val
train_data = data[:n]
val_data = data[n:]

In [ ]:
import numpy as np
block_size = 8 #or context length
train_data[:block_size+1], decode(train_data[:block_size+1].numpy())

(tensor([  0,  43,  56,  58,  67,  58,  10,   1, 107]), '\nScene. —')

In [ ]:
#example
x = train_data[:block_size] #context
y = train_data[1:block_size+1] #prediction
for t in range(block_size):
  context = x[:t+1]
  target = y[t]
  print(f"when input is {context}, the target is {target}")

when input is tensor([0]), the target is 43
when input is tensor([ 0, 43]), the target is 56
when input is tensor([ 0, 43, 56]), the target is 58
when input is tensor([ 0, 43, 56, 58]), the target is 67
when input is tensor([ 0, 43, 56, 58, 67]), the target is 58
when input is tensor([ 0, 43, 56, 58, 67, 58]), the target is 10
when input is tensor([ 0, 43, 56, 58, 67, 58, 10]), the target is 1
when input is tensor([ 0, 43, 56, 58, 67, 58, 10,  1]), the target is 107


In [ ]:
batch_size = 4 #how many independent sequences will be processed in parallel
block_size = 8 #maximum context length

def get_batch(split):
  data = train_data if split == 'train' else val_data
  ix = torch.randint(len(data) - block_size, (batch_size,)) #get a random ix with batch_size dims
  x = torch.stack([data[i:i+block_size] for i in ix]) #concatenate numbers from i to block_size for every random ix obtained
  y = torch.stack([data[i+1:i+block_size+1] for i in ix])
  return x, y

xb, yb = get_batch('train')
print(f'inputs: {xb.shape}, {xb}')
print(f'targets: {yb.shape}, {yb}')
print('---')

for b in range(batch_size):
  for t in range(block_size):
    context = xb[b, :t+1]
    target = yb[b,t]
    #print(f'when input is {context.tolist()}, the target is: {target}')
    #if t == 3:
      #break

inputs: torch.Size([4, 8]), tensor([[72,  1, 62, 66, 69, 71, 58, 72],
        [74, 71,  1, 72, 54, 64, 58,  8],
        [54, 78, 10,  1, 47, 61, 78,  1],
        [67,  1, 73, 61, 58,  1, 60, 71]])
targets: torch.Size([4, 8]), tensor([[ 1, 62, 66, 69, 71, 58, 72, 72],
        [71,  1, 72, 54, 64, 58,  8,  0],
        [78, 10,  1, 47, 61, 78,  1, 57],
        [ 1, 73, 61, 58,  1, 60, 71, 54]])
---


# Bigram

In [ ]:
#ahora procesamos todo en una red neuronal
import torch.nn as nn
from torch.nn import functional as F

class BigramLanguageModel(nn.Module):
  def __init__(self, vocab_size):
    super().__init__()
    # each token reads off the logits for the next token from a lookup table
    self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)

  def forward(self, idx, targets=None):
    #idx and targets are both (B,T) tensor of integers
    logits = self.token_embedding_table(idx) # (B,T,C) batch, time, channels = 4, 8 (block_size), 112 (vocab_size)

    if targets == None:
      loss = None
    else:
      B, T, C = logits.shape
      logits = logits.view(B*T, C) #cambiamos el orden por cómo funciona F.cross_entropy
      targets = targets.view(B*T)
      loss = F.cross_entropy(logits, targets) # nll
    return logits, loss

  def generate(self, idx, max_new_tokens):
    #idx is (B,T) array of indices: xb
    for _ in range(max_new_tokens):
      #get the predictions
      logits, loss = self(idx)
      #focus only on the last time step
      logits = logits[:, -1, :] #selects the -1 (last) element of the T dimension because that's the next character, becomes (B,C)
      #softmax
      probs = F.softmax(logits, dim=-1) # (B,C)
      #sample from distribution
      idx_next = torch.multinomial(probs, num_samples=1) # (B,1)
      #append sampled index to running sequence
      idx = torch.cat((idx, idx_next), dim=1) # (B,T+1)
    return idx

m = BigramLanguageModel(vocab_size)
logits, loss = m(xb, yb)
print(logits.shape, loss)

torch.Size([32, 112]) tensor(5.2095, grad_fn=<NllLossBackward0>)


In [ ]:
print(decode(m.generate(idx = torch.zeros((1,1), dtype=torch.long), max_new_tokens=100)[0].tolist()))


èoQ/gApœAqJç'ÈÈçxáF4uAyé*7!kâ*75ïèTäkfbö’xSrë;]Æywm0RÉx!N!ecè[œóÀBhîSûYéqé\Epn’lœM£!d- a'ûn4I)P0fÈ9E


In [ ]:
optimizer = torch.optim.AdamW(m.parameters(), lr=1e-3)

In [ ]:
batch_size = 32
for steps in range(10000):
  #sample a batch
  xb, yb = get_batch('train')

  #evaluate the loss
  logits, loss = m(xb, yb)
  optimizer.zero_grad(set_to_none=True)
  loss.backward()
  optimizer.step()


print(loss.item())

2.411374568939209


In [ ]:
print(decode(m.generate(idx = torch.zeros((1,1), dtype=torch.long), max_new_tokens=200)[0].tolist()))






Wa les, l thelvearel s afone rexqSpes isitachave Jan, be Ithat nddoot spr!” g, is inide thed sosorthetrstheavemind tolas quthals omeareks, l ckntwongacoyol ay s ricodast t owinin, Bubonkeencouting


# Attention
## masking

In [ ]:
B, T, C = 4, 8, 2 # batch, time, channels
x = torch.randn(B, T, C)
x.shape, x

(torch.Size([4, 8, 2]), tensor([[[ 1.0694,  0.7632],
          [-0.2685, -0.8615],
          [-0.0648,  1.3087],
          [-2.0130,  1.4452],
          [ 1.3777, -0.9225],
          [-0.2851, -0.6933],
          [ 0.8647, -0.4860],
          [-1.2882, -0.3192]],
 
         [[-2.7493, -0.8327],
          [ 1.5506, -0.8510],
          [-0.5497,  0.1812],
          [ 1.6403, -0.8118],
          [ 0.0643,  1.0898],
          [ 1.7347,  0.3218],
          [ 0.4110, -2.6798],
          [ 0.3078, -0.9501]],
 
         [[ 1.1694,  0.8299],
          [-2.4168, -0.7675],
          [-0.5913, -0.0316],
          [ 0.5090,  0.5215],
          [ 0.6991,  0.2643],
          [ 2.5257, -0.0798],
          [ 0.7582, -0.3698],
          [ 1.8004, -1.9457]],
 
         [[ 0.2079, -0.0963],
          [-1.2654, -1.4188],
          [ 0.3596,  1.0491],
          [ 0.4066,  1.0815],
          [ 0.5730,  0.9116],
          [ 0.1121, -0.4333],
          [ 0.3308, -0.2331],
          [-0.7198,  1.0286]]]))

In [ ]:
# we want x[b,t] = mean_{i <= t} x[b,i]
xbow = torch.zeros((B,T,C)) #bag of words, average
for b in range(B): # iterate over batch
  for t in range(T): # iterate over "time"
    xprev = x[b, :t+1] # (t, C)
    xbow[b,t] = torch.mean(xprev, 0) #average dimension 0: t

x[0]

tensor([[ 1.0694,  0.7632],
        [-0.2685, -0.8615],
        [-0.0648,  1.3087],
        [-2.0130,  1.4452],
        [ 1.3777, -0.9225],
        [-0.2851, -0.6933],
        [ 0.8647, -0.4860],
        [-1.2882, -0.3192]])

In [ ]:
xbow[0] # every row is the average of itself and its preceding rows

tensor([[ 1.0694,  0.7632],
        [ 0.4004, -0.0492],
        [ 0.2454,  0.4035],
        [-0.3192,  0.6639],
        [ 0.0202,  0.3466],
        [-0.0307,  0.1733],
        [ 0.0972,  0.0791],
        [-0.0760,  0.0293]])

In [ ]:
a = torch.ones(3, 3)
b = torch.randint(0, 10, (3,2)).float()
c = a @ b
print(f'a = {a}')
print('---')
print(f'b = {b}')
print('---')
print(f'c = {c}')
print('---')

a = tensor([[1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.]])
---
b = tensor([[4., 0.],
        [7., 2.],
        [9., 8.]])
---
c = tensor([[20., 10.],
        [20., 10.],
        [20., 10.]])
---


In [ ]:
a = torch.tril(torch.ones(3, 3))
a = a / torch.sum(a, 1, keepdim=True)
b = torch.randint(0, 10, (3,2)).float()
c = a @ b
print(f'a = {a}')
print('---')
print(f'b = {b}')
print('---')
print(f'c = {c}')
print('---')

a = tensor([[1.0000, 0.0000, 0.0000],
        [0.5000, 0.5000, 0.0000],
        [0.3333, 0.3333, 0.3333]])
---
b = tensor([[5., 7.],
        [7., 2.],
        [4., 3.]])
---
c = tensor([[5.0000, 7.0000],
        [6.0000, 4.5000],
        [5.3333, 4.0000]])
---


In [ ]:
wei = torch.tril(torch.ones(T, T))
wei = wei / wei.sum(1, keepdim=True)
wei

tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.5000, 0.5000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.3333, 0.3333, 0.3333, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2500, 0.2500, 0.2500, 0.2500, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2000, 0.2000, 0.2000, 0.2000, 0.2000, 0.0000, 0.0000, 0.0000],
        [0.1667, 0.1667, 0.1667, 0.1667, 0.1667, 0.1667, 0.0000, 0.0000],
        [0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.0000],
        [0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250]])

In [ ]:
xbow2 = wei @ x 
torch.allclose(xbow, xbow2)

True

In [ ]:
#version 3 
tril = torch.tril(torch.ones(T, T)) # triangular matrix
wei = torch.zeros((T,T)) # all zeros
tril, wei

(tensor([[1., 0., 0., 0., 0., 0., 0., 0.],
         [1., 1., 0., 0., 0., 0., 0., 0.],
         [1., 1., 1., 0., 0., 0., 0., 0.],
         [1., 1., 1., 1., 0., 0., 0., 0.],
         [1., 1., 1., 1., 1., 0., 0., 0.],
         [1., 1., 1., 1., 1., 1., 0., 0.],
         [1., 1., 1., 1., 1., 1., 1., 0.],
         [1., 1., 1., 1., 1., 1., 1., 1.]]),
 tensor([[0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0.]]))

donde `tril` sea 0, convertir eso en `-inf`. eso hace que los tokens del futuro no puedan comunicarse con los del pasado.

In [ ]:
wei = wei.masked_fill(tril == 0, float('-inf'))
wei

tensor([[0., -inf, -inf, -inf, -inf, -inf, -inf, -inf],
        [0., 0., -inf, -inf, -inf, -inf, -inf, -inf],
        [0., 0., 0., -inf, -inf, -inf, -inf, -inf],
        [0., 0., 0., 0., -inf, -inf, -inf, -inf],
        [0., 0., 0., 0., 0., -inf, -inf, -inf],
        [0., 0., 0., 0., 0., 0., -inf, -inf],
        [0., 0., 0., 0., 0., 0., 0., -inf],
        [0., 0., 0., 0., 0., 0., 0., 0.]])

In [ ]:
wei = F.softmax(wei, dim=-1) #softmax sobre cada fila, normalizando wei
wei

tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.5000, 0.5000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.3333, 0.3333, 0.3333, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2500, 0.2500, 0.2500, 0.2500, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2000, 0.2000, 0.2000, 0.2000, 0.2000, 0.0000, 0.0000, 0.0000],
        [0.1667, 0.1667, 0.1667, 0.1667, 0.1667, 0.1667, 0.0000, 0.0000],
        [0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.0000],
        [0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250]])

In [ ]:
xbow3 = wei @ x
torch.allclose(xbow, xbow3)

True

## self-attention

In [ ]:
B, T, C = 4, 8, 32 # batch, time, channels
x = torch.randn(B, T, C)

# a single Head performing self-attention
head_size = 16
key = nn.Linear(C, head_size, bias=False)
query = nn.Linear(C, head_size, bias=False)
value = nn.Linear(C, head_size, bias=False)
k = key(x) # (B, T, 16)
q = query(x) # (B, T, 16)
wei = q @ k.transpose(-2, -1) # (B, T, 16) @ (B, 16, T) ---> (B, T, T)

tril = torch.tril(torch.ones(T,T))
#wei = torch.zeros((T,T))
wei = wei.masked_fill_(tril == 0, float('-inf'))
wei = F.softmax(wei, dim=-1)

v = value(x)
#out = wei @ x
out = wei @ v

out.shape

torch.Size([4, 8, 16])

In [ ]:
wei

tensor([[[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.6574, 0.3426, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.0284, 0.8113, 0.1602, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.1182, 0.5935, 0.2381, 0.0501, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.2200, 0.3199, 0.0587, 0.2337, 0.1677, 0.0000, 0.0000, 0.0000],
         [0.2118, 0.0444, 0.0539, 0.3532, 0.0216, 0.3151, 0.0000, 0.0000],
         [0.8061, 0.0073, 0.0463, 0.0455, 0.0840, 0.0077, 0.0030, 0.0000],
         [0.5191, 0.0317, 0.0329, 0.1550, 0.0481, 0.0541, 0.0514, 0.1076]],

        [[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.1941, 0.8059, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.0323, 0.5310, 0.4367, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.0982, 0.3470, 0.5172, 0.0376, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.0212, 0.0745, 0.0153, 0.8860, 0.0029, 0.0000, 0.0000, 0.0000],
         [0.3300, 0.012

#Scripts

In [ ]:
#%%writefile bigram.py

import torch
import torch.nn as nn
from torch.nn import functional as F

# hyperparameters
batch_size = 32 # how many independent sequences will we process in parallel?
block_size = 8 # what is the maximum context length for predictions?
max_iters = 3000
eval_interval = 300
learning_rate = 1e-2
device = 'cuda' if torch.cuda.is_available() else 'cpu'
eval_iters = 200
# ------------

# dataset
# !wget https://github.com/DanteNoguez/gpt-wilde/raw/main/wilde.txt
with open('wilde.txt', 'r', encoding='utf-8') as f:
    text = f.read()

# here are all the unique characters that occur in this text
chars = sorted(list(set(text)))
vocab_size = len(chars)
# create a mapping from characters to integers
stoi = { ch:i for i,ch in enumerate(chars) }
itos = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [stoi[c] for c in s] # encoder: take a string, output a list of integers
decode = lambda l: ''.join([itos[i] for i in l]) # decoder: take a list of integers, output a string

# Train and test splits
data = torch.tensor(encode(text), dtype=torch.long)
n = int(0.9*len(data)) # first 90% will be train, rest val
train_data = data[:n]
val_data = data[n:]

# data loading
def get_batch(split):
    # generate a small batch of data of inputs x and targets y
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y

@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

# super simple bigram model
class BigramLanguageModel(nn.Module):

    def __init__(self, vocab_size):
        super().__init__()
        # each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)

    def forward(self, idx, targets=None):

        # idx and targets are both (B,T) tensor of integers
        logits = self.token_embedding_table(idx) # (B,T,C)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # get the predictions
            logits, loss = self(idx)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx

model = BigramLanguageModel(vocab_size)
m = model.to(device)

# create a PyTorch optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

for iter in range(max_iters):

    # every once in a while evaluate the loss on train and val sets
    if iter % eval_interval == 0:
        losses = estimate_loss()
        print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")

    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = model(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

# generate from the model
context = torch.zeros((1, 1), dtype=torch.long, device=device)
print(decode(m.generate(context, max_new_tokens=500)[0].tolist()))

step 0: train loss 5.2350, val loss 5.2418
step 300: train loss 2.9223, val loss 2.9089
step 600: train loss 2.5557, val loss 2.5607
step 900: train loss 2.5037, val loss 2.4935
step 1200: train loss 2.4759, val loss 2.4697
step 1500: train loss 2.4697, val loss 2.4662
step 1800: train loss 2.4473, val loss 2.4514
step 2100: train loss 2.4643, val loss 2.4447
step 2400: train loss 2.4507, val loss 2.4519
step 2700: train loss 2.4533, val loss 2.4446


“Wonomis te I ghe d oyedeonde t.” ovehed A atou Marinipined m mortsecove testh. m? t My hty maythichenaulinishe hee’thy g ol re rine, ngin ARea Anodlpé
He ind har ar thiven’
Mrugi, me fefreg. iche cce”
Worely l t abo s andeeed aron hir ondory hy me t ve d, hel acowant t ason, And Indebloy. by, o hen. a I bllid ary, sis7&î7westhass. ventwiore. s He f t werthol s l the mee o, I s t frkily, ps, he lomyo f le—




VI ove buthes. us, uldolarpt theld ceut Thel; re Wetharelpáedeard. os.

LD the r t: a


In [23]:
!nvidia-smi

Sat Jan 21 03:17:02 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  A100-SXM4-40GB      Off  | 00000000:00:04.0 Off |                    0 |
| N/A   29C    P0    50W / 400W |   8368MiB / 40536MiB |      0%      Default |
|                               |                      |             Disabled |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
# %%writefile gpt.py

import torch
import torch.nn as nn
from torch.nn import functional as F

# hyperparameters
batch_size = 64 # how many independent sequences will we process in parallel?
block_size = 256 # what is the maximum context length for predictions?
max_iters = 5000
eval_interval = 500
learning_rate = 3e-4
device = 'cuda' if torch.cuda.is_available() else 'mps'
eval_iters = 200
n_embd = 512 # number of embeddings
n_head = 8
n_layer = 8
dropout = 0.2
# ------------

# dataset
# !wget https://github.com/DanteNoguez/gpt-wilde/raw/main/wilde.txt
with open('wilde.txt', 'r', encoding='utf-8') as f:
    text = f.read()

# here are all the unique characters that occur in this text
chars = sorted(list(set(text)))
vocab_size = len(chars)
# create a mapping from characters to integers
stoi = { ch:i for i,ch in enumerate(chars) }
itos = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [stoi[c] for c in s] # encoder: take a string, output a list of integers
decode = lambda l: ''.join([itos[i] for i in l]) # decoder: take a list of integers, output a string

# Train and test splits
data = torch.tensor(encode(text), dtype=torch.long)
n = int(0.9*len(data)) # first 90% will be train, rest val
train_data = data[:n]
val_data = data[n:]

# data loading
def get_batch(split):
    # generate a small batch of data of inputs x and targets y
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y

@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

# attention
class Head(nn.Module):
    def __init__(self, head_size):
        super().__init__()
        self.key = nn.Linear(n_embd, head_size, bias=False)
        self.query = nn.Linear(n_embd, head_size, bias=False)
        self.value = nn.Linear(n_embd, head_size, bias=False)
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        B,T,C = x.shape
        k = self.key(x)
        q = self.query(x)
        #compute attention scores ("affinities")
        wei = q @ k.transpose(-2, -1) * C**0.5 # (B,T,C) @ (B,C,T) -> (B,T,T)
        wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf')) # B,T,T
        wei = F.softmax(wei, dim=-1) # (B,T,T)
        wei = self.dropout(wei)
        # perform the weighted aggregation of values
        v = self.value(x) # (B,T,C)
        out = wei @ v
        return out

class MultiHeadAttention(nn.Module):
    def __init__(self, num_heads, head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
        self.proj = nn.Linear(n_embd, n_embd)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        out = torch.cat([h(x) for h in self.heads], dim=-1)
        out = self.dropout(self.proj(out))
        return out

# FFN
class FeedForward(nn.Module):
    def __init__(self, n_embd):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embd, 4 * n_embd), # multiply by 4 like in the paper
            nn.ReLU(),
            nn.Linear(4 * n_embd, n_embd),
            nn.Dropout(dropout),
        )

    def forward(self, x):
        return self.net(x)

# LayerNorm
# Same as batch norm but without buffers, calculating means and std across rows instead of columns, no momentum
# and no worries about training or test time
class LayerNorm(nn.Module):
    def __init__(self, dim, epsilon=1e-5):
        self.epsilon = epsilon
        self.gamma = torch.ones(dim)
        self.beta = torch.zeros(dim)

    def __call__(self, x):
        xmean = x.mean(1, keepdim=True) # across rows
        xvar = x.var(1, keepdim=True)
        xhat = (x - xmean) / torch.sqrt(xvar + self.eps)
        self.out = self.gamma * xhat + self.beta
        return self.out

    def parameters(self):
        return [self.gamma, self.beta]

class Block(nn.Module):
    # transformer block: communication followed by computation"
    def __init__(self, n_embd, n_head):
        super().__init__()
        head_size = n_embd // n_head
        self.sa = MultiHeadAttention(n_head, head_size)
        self.ffn = FeedForward(n_embd)
        self.ln1 = nn.LayerNorm(n_embd)
        self.ln2 = nn.LayerNorm(n_embd)

    def forward(self, x):
        x = x + self.sa(self.ln1(x)) # sum with x is residual connections
        x = x + self.ffn(self.ln2(x)) # pre-normalization is new, not equal to the paper
        return x

# super simple transformer (decoder only)
class Transformer(nn.Module):

    def __init__(self):
        super().__init__()
        # each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
        self.position_embedding_table = nn.Embedding(block_size, n_embd)
        # self.sa_head = Head(n_embd)
        # self.sa_heads = MultiHeadAttention(6, n_embd//6) # 6 heads 
        self.blocks = nn.Sequential(*[Block(n_embd, n_head=n_head) for _ in range(n_layer)])
        self.ln_f = nn.LayerNorm(n_embd)
        #self.ffn = FeedForward(n_embd)
        self.lm_head = nn.Linear(n_embd, vocab_size) # language modeling head

    def forward(self, idx, targets=None):
        B, T = idx.shape

        # idx and targets are both (B,T) tensor of integers
        tok_emb = self.token_embedding_table(idx) # (B,T,C)
        pos_embd = self.position_embedding_table(torch.arange(T, device=device)) # (T,C)
        x = tok_emb + pos_embd # (B,T,C)
        # x = self.sa_heads(x)
        x = self.blocks(x) # aply one head of self-attention (B,T,C)
        # x = self.ffn(x)
        x = self.ln_f(x)
        logits = self.lm_head(x) # (B,T,vocab_size)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # crop idx to the last block_size tokens
            idx_cond = idx[:, -block_size:]
            # get the predictions
            logits, loss = self(idx_cond) 
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx

model = Transformer()
m = model.to(device)

# create a PyTorch optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

for iter in range(max_iters):

    # every once in a while evaluate the loss on train and val sets
    if iter % eval_interval == 0:
        losses = estimate_loss()
        print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")

    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = model(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

# generate from the model
context = torch.zeros((1, 1), dtype=torch.long, device=device)
print(decode(m.generate(context, max_new_tokens=500)[0].tolist()))

In [21]:
print(decode(m.generate(context, max_new_tokens=500)[0].tolist()))



H

ANESil tined pinowhamppl bing cepe. r weroulitope chisur ttr enfr alos aveis, e than t or incak, f dofetigono orsppoung,’s w s whelis mee-u, s ithuthis It s Heladres se ound ir me bulyo cac t t juanghicisby w halil ts ld sind, hathang I tir s t oly Ceend we d We a the pr tusuns wnind chass ar thang. g. he, lls cayoorsupr.

I’se ascknat othe nd if ss ‘Creu stimelid t mid ange hose o linoned geeas f lf I wocatles he imand teres seroff ckinours arthall APoneved the (e ed oou wint blinowin Dan b


In [22]:
print(sum(p.numel() for p in m.parameters())/1e6, 'M parameters')

25.45368 M parameters


In [3]:
import torch
import torch.nn as nn
from torch.nn import functional as F

# hyperparameters
batch_size = 64 # how many independent sequences will we process in parallel?
block_size = 256 # what is the maximum context length for predictions?
max_iters = 5000
eval_interval = 500
learning_rate = 3e-4
device = 'cuda' if torch.cuda.is_available() else 'cpu'
eval_iters = 200
n_embd = 512
n_head = 8
n_layer = 8
dropout = 0.2
# ------------

torch.manual_seed(1337)

# wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
with open('wilde.txt', 'r', encoding='utf-8') as f:
    text = f.read()

# here are all the unique characters that occur in this text
chars = sorted(list(set(text)))
vocab_size = len(chars)
# create a mapping from characters to integers
stoi = { ch:i for i,ch in enumerate(chars) }
itos = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [stoi[c] for c in s] # encoder: take a string, output a list of integers
decode = lambda l: ''.join([itos[i] for i in l]) # decoder: take a list of integers, output a string

# Train and test splits
data = torch.tensor(encode(text), dtype=torch.long)
n = int(0.9*len(data)) # first 90% will be train, rest val
train_data = data[:n]
val_data = data[n:]

# data loading
def get_batch(split):
    # generate a small batch of data of inputs x and targets y
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y

@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

class Head(nn.Module):
    """ one head of self-attention """

    def __init__(self, head_size):
        super().__init__()
        self.key = nn.Linear(n_embd, head_size, bias=False)
        self.query = nn.Linear(n_embd, head_size, bias=False)
        self.value = nn.Linear(n_embd, head_size, bias=False)
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))

        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        B,T,C = x.shape
        k = self.key(x)   # (B,T,C)
        q = self.query(x) # (B,T,C)
        # compute attention scores ("affinities")
        wei = q @ k.transpose(-2,-1) * C**-0.5 # (B, T, C) @ (B, C, T) -> (B, T, T)
        wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf')) # (B, T, T)
        wei = F.softmax(wei, dim=-1) # (B, T, T)
        wei = self.dropout(wei)
        # perform the weighted aggregation of the values
        v = self.value(x) # (B,T,C)
        out = wei @ v # (B, T, T) @ (B, T, C) -> (B, T, C)
        return out

class MultiHeadAttention(nn.Module):
    """ multiple heads of self-attention in parallel """

    def __init__(self, num_heads, head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
        self.proj = nn.Linear(n_embd, n_embd)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        out = torch.cat([h(x) for h in self.heads], dim=-1)
        out = self.dropout(self.proj(out))
        return out

class FeedFoward(nn.Module):
    """ a simple linear layer followed by a non-linearity """

    def __init__(self, n_embd):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embd, 4 * n_embd),
            nn.ReLU(),
            nn.Linear(4 * n_embd, n_embd),
            nn.Dropout(dropout),
        )

    def forward(self, x):
        return self.net(x)

class Block(nn.Module):
    """ Transformer block: communication followed by computation """

    def __init__(self, n_embd, n_head):
        # n_embd: embedding dimension, n_head: the number of heads we'd like
        super().__init__()
        head_size = n_embd // n_head
        self.sa = MultiHeadAttention(n_head, head_size)
        self.ffwd = FeedFoward(n_embd)
        self.ln1 = nn.LayerNorm(n_embd)
        self.ln2 = nn.LayerNorm(n_embd)

    def forward(self, x):
        x = x + self.sa(self.ln1(x))
        x = x + self.ffwd(self.ln2(x))
        return x

# super simple bigram model
class BigramLanguageModel(nn.Module):

    def __init__(self):
        super().__init__()
        # each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
        self.position_embedding_table = nn.Embedding(block_size, n_embd)
        self.blocks = nn.Sequential(*[Block(n_embd, n_head=n_head) for _ in range(n_layer)])
        self.ln_f = nn.LayerNorm(n_embd) # final layer norm
        self.lm_head = nn.Linear(n_embd, vocab_size)

    def forward(self, idx, targets=None):
        B, T = idx.shape

        # idx and targets are both (B,T) tensor of integers
        tok_emb = self.token_embedding_table(idx) # (B,T,C)
        pos_emb = self.position_embedding_table(torch.arange(T, device=device)) # (T,C)
        x = tok_emb + pos_emb # (B,T,C)
        x = self.blocks(x) # (B,T,C)
        x = self.ln_f(x) # (B,T,C)
        logits = self.lm_head(x) # (B,T,vocab_size)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # crop idx to the last block_size tokens
            idx_cond = idx[:, -block_size:]
            # get the predictions
            logits, loss = self(idx_cond)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx

model = BigramLanguageModel()
m = model.to(device)
# print the number of parameters in the model
print(sum(p.numel() for p in m.parameters())/1e6, 'M parameters')

# create a PyTorch optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

for iter in range(max_iters):

    # every once in a while evaluate the loss on train and val sets
    if iter % eval_interval == 0 or iter == max_iters - 1:
        losses = estimate_loss()
        print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")

    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = model(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

# generate from the model
context = torch.zeros((1, 1), dtype=torch.long, device=device)
print(decode(m.generate(context, max_new_tokens=500)[0].tolist()))

25.45368 M parameters
step 0: train loss 4.9631, val loss 4.9682
step 500: train loss 2.0226, val loss 2.0326
step 1000: train loss 1.5666, val loss 1.5973
step 1500: train loss 1.3837, val loss 1.4434
step 2000: train loss 1.2860, val loss 1.3601
step 2500: train loss 1.2232, val loss 1.3170
step 3000: train loss 1.1799, val loss 1.2904
step 3500: train loss 1.1445, val loss 1.2655
step 4000: train loss 1.1132, val loss 1.2527
step 4500: train loss 1.0882, val loss 1.2383
step 4999: train loss 1.0648, val loss 1.2267


CHEVELEY. Ah! I red it for you in everything.”

Hallward, cannot be the picture, girled over success with me as nothing of the photogramb’s, though of temper to a seal complete you have too ears. But Partiae is telling to Basil, a heart-fiul face did not him, an enthusiastic blue of Southeric and struggled. Five times of flaunt and môid by fall of wildling apprehends, and thus to see it from Brusalium and hath dagges Wedribe — of Common mistakes it as beautiful, that se

In [4]:
from pathlib import Path

def save_model(model: torch.nn.Module, target_dir: str, model_name: str):
  """Saves a PyTorch model to a target directory"""
  target_dir_path = Path(target_dir)
  target_dir_path.mkdir(parents=True, exist_ok=True)

  assert model_name.endswith('.pth') or model_name.endswith('.pt'), 'model_name should end with .pt or .pth'
  model_save_path = target_dir_path / model_name

  print(f'Saving model to: {model_save_path}.')
  torch.save(obj=model.state_dict(),
             f=model_save_path)

In [5]:
save_model(m, 'models', 'gpt-wilde.pth')

Saving model to: models/gpt-wilde.pth.


In [6]:
open('more.txt', 'w').write(decode(m.generate(context, max_new_tokens=10000)[0].tolist()))

10001